In [68]:
import numpy as np
import itertools
from scipy.signal import convolve2d

In [258]:
class Board():
    def __init__(self):
        self.ROWS, self.COLUMNS, self.connect = 3, 3, 3
        self.state = np.zeros(shape=(self.ROWS*3,self.COLUMNS*3))
        self.meta_state = np.zeros(shape=(self.ROWS, self.COLUMNS))

        #setup kernals
        self.horizontal_kernel = np.ones(shape=(1,self.connect))
        self.vertical_kernel = np.transpose(self.horizontal_kernel)
        self.diag1_kernel = np.eye(self.connect, dtype=np.uint8)
        self.diag2_kernel = np.fliplr(self.diag1_kernel)
        self.detection_kernels = [self.horizontal_kernel, self.vertical_kernel, self.diag1_kernel, self.diag2_kernel]
        
    def get_available_moves(self):
        if (self.state==0).all():
            return list(itertools.product(range(self.ROWS*self.COLUMNS), repeat=2))
        x, y = self.convert_coord(tuple(np.argwhere(np.absolute(self.state)==2)[0]))
        minigame = self.state[x*3:(x+1)*3, y*3:(y+1)*3]
        if self.minigame_is_over((x,y)):
            #do every other miniboard
            boards = list(np.argwhere(self.meta_state==0))
            possible_moves = []
            for board in boards:
                x, y = board[0], board[1]
                minigame = self.state[x*3:(x+1)*3, y*3:(y+1)*3]
                moves = list(np.argwhere(minigame==0))
                moves = [(m[0]+(x*3), m[1]+(y*3)) for m in moves]
                possible_moves.extend(moves)
        else:
            possible_moves = list(np.argwhere(minigame==0))
            possible_moves = [(m[0]+(x*3), m[1]+(y*3)) for m in possible_moves]
            
        return possible_moves
        
    def make_move(self, move, player): #player is either 1 or -1
        if move not in self.get_available_moves():
            raise ValueError('INVALID MOVE YOU DUMB BUTT!')
        #change the last move to not the most recent
        self.state[np.absolute(self.state)==2] /= 2
        
        #update new move
        self.state[move[0], move[1]] = 2*player
        
        #update metaboard if needed
        meta_coord = self.convert_coord(move)
        minigame_result = self.minigame_is_over(meta_coord)
        minigame = np.copy(self.state[meta_coord[0]*3:(meta_coord[0]+1)*3, meta_coord[1]*3:(meta_coord[1]+1)*3])
        minigame[np.absolute(minigame)==2] /=2
        if minigame_result != 0:
            self.meta_state[meta_coord[0], meta_coord[1]] = minigame_result        
    
    def minigame_is_over(self, coord): #takes minigame coord
        x, y = coord
        minigame = np.copy(self.state[x*3:(x+1)*3, y*3:(y+1)*3])
        minigame[np.absolute(minigame)==2] /=2
        return self.check_convolve(minigame)
    
    def is_game_over(self):
        return self.check_convolve(self.meta_state)
    
    def check_convolve(self, game):
        for kernel in self.detection_kernels:
            if (convolve2d(game == 1, kernel, mode="valid") == self.connect).any():
                return 1
    
        for kernel in self.detection_kernels:
            if (convolve2d(game == -1, kernel, mode="valid") == self.connect).any():
                return -1
            
        if (game != 0).all():
            return 1e-4 #full and draw
        
        return 0 #can still play
    
    @staticmethod
    def convert_coord(coord):
        x, y = coord
        return x%3, y%3
    
    def display(self):
        pieces = {
            0: ' ',
            1: 'x',
            -1: 'o',
            2: 'X',
            -2: 'O'
        }
        for i, row in enumerate(self.state):
            l = '|'
            for j, piece in enumerate(row):
                l += f'{pieces[piece]}|'
                l += '|' if j==2 or j==5 else ''
            if i==3 or i==6:
                print('='*21)
            print(l)

In [250]:
b = Board()

In [251]:
b.display()

| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |


In [252]:
b.make_move((3,0), -1)

In [253]:
b.get_available_moves()

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [259]:
outcomes = []
for _ in range(100):
    b = Board()
    player = 1
    while b.is_game_over() == 0:
        moves = b.get_available_moves()
        if len(moves) == 0:
            print(b.meta_state)
        move = moves[np.random.randint(0, len(moves))]
        b.make_move(move, player)
        player *= -1
        b.display()
        print('')
        print('')
    #     print(b.meta_state)
    #     print('')
    #     print('')
    outcomes.append(b.is_game_over())

| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | |X|
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |


| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | |x|
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | |O|
| | | || | | || | | |
| | | || | | || | | |


| | | || | | || |X| |
| | | || | | || | | |
| | | || | | || | |x|
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | |o|
| | | || | | || | | |
| | | || | | || | | |


| | | || |O| || |x| |
| | | || | | || | | |
| | | || | | || | |x|
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | |o|
| | | || | | || | | |
| | | || | | || | | |


| | | ||X|o| || |x| |
| | | || | | || | | |
| | | || | | || | |x|
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | | |
| | | || | | || | |o|
| | | || | | || | | |
| | | || | | || | | |




|o|o|x||x|o|x||o|x| |
|x|x|x|| |x| ||o|x| |
|x| |o||x|o| ||o| |x|
|o|o|x||o|x|o||o|x|o|
|x|o|o||x|x|o||o|O|x|
|o|o|x||x|o|x||x|x|x|
|x|x|o||x|o|x|| |o|o|
|o|o|o||x|o|x||x|o| |
|o|o|x||x|x|o||x|o|o|


[[ 1.e+00  1.e+00 -1.e+00]
 [ 0.e+00  1.e-04  0.e+00]
 [-1.e+00  1.e+00 -1.e+00]]


ValueError: Range cannot be empty (low >= high) unless no samples are taken

In [245]:
b.get_available_moves()

[0 1]
[[0, 0], [0, 2], [2, 0]]
[(0, 3), (0, 5), (2, 3)]
[0 2]
[[0, 1], [0, 2], [1, 2]]
[(0, 7), (0, 8), (1, 8)]
[1 2]
[[0, 2], [1, 0]]
[(3, 8), (4, 6)]


[(0, 3), (0, 5), (2, 3), (0, 7), (0, 8), (1, 8), (3, 8), (4, 6)]